In [1]:
import os
import numpy as np
import pandas as pd
import sqlite3
import math
import sys

from astropy.coordinates import SkyCoord
import astropy.units as u
from collections import OrderedDict as Odict
from astropy import time

import lsst.afw.geom as afwGeom
import lsst.afw.cameraGeom
import lsst.geom as geom
import lsst.afw.display as afwDisplay
from lsst.daf.persistence import Butler


from lsst.afw.geom import makeSkyWcs
from lsst.obs.lsst.imsim import ImsimMapper

from lsst.sims.utils import angularSeparation
from lsst.sims.utils import getRotSkyPos
from lsst.sims.catUtils import supernovae
from lsst.sims.catUtils.supernovae import SNObject
from lsst.sims.catUtils.utils import ObservationMetaDataGenerator
from lsst.sims.catUtils.dust import EBVbase
from lsst.sims.photUtils.Sed import Sed
from lsst.sims.photUtils.BandpassDict import BandpassDict
from lsst.sims.photUtils.SignalToNoise import calcSNR_m5, calcMagError_m5
from lsst.sims.photUtils.PhotometricParameters import PhotometricParameters

sys.path.insert(0, '/global/homes/r/rhlozek/gcr-catalogs')
import GCRCatalogs

repo = '/global/cscratch1/sd/rearmstr/new_templates/diffim_template'
butler = Butler(repo)
diaSrc = GCRCatalogs.load_catalog('dc2_dia_source_run1.2p_test')
diaObject = GCRCatalogs.load_catalog('dc2_dia_object_run1.2p_test')
truth_cat = GCRCatalogs.load_catalog('dc2_truth_run1.2_variable_summary')
truth_lc = GCRCatalogs.load_catalog('dc2_truth_run1.2_variable_lightcurve')

/opt/lsst/software/stack/stack/miniconda3-4.5.12-1172c30/Linux64/sims_catalogs/2.13.0.sims-3-g60469dd+6/python/lsst/sims/catalogs/db/dbConnection.py:555: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/opt/lsst/software/stack/stack/miniconda3-4.5.12-1172c30/Linux64/sims_catalogs/2.13.0.sims-3-g60469dd+6/python/lsst/sims/catalogs/db/dbConnection.py:555: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/opt/lsst/software/stack/stack/miniconda3-4.5.12-1172c30/Linux64/sims_catalogs/2.13.0.sims-3-g60469dd+6/python/lsst/sims/catalogs/db/dbConnection.py:555: UserWarning: Duplicate object type id 4 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally u

In [2]:
# Defining the box from the truth catalog
tract = 4849
patch = (6, 6)

# Cut a piece of the sky from the relevant tract
skymap = butler.get('deepCoadd_skyMap')
tract_info = skymap[tract]

#patchpoly = tract_info.getPatchInfo(patch)
#bar = patchpoly.getOuterSkyPolygon(tract_info.getWcs())

tract_box = afwGeom.Box2D(tract_info.getBBox())
tract_pos_list = tract_box.getCorners()
wcs = tract_info.getWcs()
corners = wcs.pixelToSky(tract_pos_list)
corners = np.array([[c.getRa().asDegrees(), c.getDec().asDegrees()] for c in corners])
ra = corners[:, 0]
dec = corners[:, 1]
min_ra, max_ra = np.min(ra), np.max(ra)
min_dec, max_dec = np.min(dec), np.max(dec)
print( 'boundaries for simulation tract:', min_ra, max_ra, min_dec, max_dec)


boundaries for simulation tract: 52.93572544959448 54.72776196140405 -29.78294629153197 -28.2276731919467


In [3]:
area_cut = [f'ra > {min_ra}', f'ra < {max_ra}', f'dec > {min_dec}', f'dec < {max_dec}']
sn_cut = ['sn == 1']
all_cuts = area_cut + sn_cut
# getting info from the DC2 catalogs
columns = ['ra', 'dec', 'redshift', 'uniqueId', 'galaxy_id', 'sn']
truth_sne = pd.DataFrame(truth_cat.get_quantities(columns, filters=all_cuts))
i = 10
sn = truth_sne.iloc[0]
ra, dec = sn['ra'], sn['dec']
print(ra, dec)

# Match on RA, Dec
sn_position = SkyCoord(sn['ra'], sn['dec'], unit='deg')

diaObject_cat = diaObject.get_quantities(['ra', 'dec', 'diaObjectId'])
diaObject_positions = SkyCoord(diaObject_cat['ra'], diaObject_cat['dec'], unit='deg')
print(diaObject_positions)
idx, sep2d, _ = sn_position.match_to_catalog_sky(diaObject_positions)
print(f'Index: {idx} is {sep2d.to(u.arcsec)[0]:0.6f} away')
diaObjectId = diaObject_cat['diaObjectId'][idx]

53.00727812651823 -28.387552632959427
<SkyCoord (ICRS): (ra, dec) in deg
    [(53.19006979, -28.42791565), (53.17706282, -28.42526992),
     (53.17008481, -28.41005509), ..., (52.99593198, -28.33793884),
     (53.00038471, -28.33280359), (53.07184798, -28.28804726)]>
Index: 22 is 0.028836 arcsec away


In [7]:
# creating the telescope camera objects
mapper = ImsimMapper()
camera = mapper.camera
trans = [detector.getTransform(lsst.afw.cameraGeom.cameraSys.PIXELS,
         lsst.afw.cameraGeom.cameraSys.FIELD_ANGLE) for detector in camera]
boxes = [detector.getBBox() for detector in camera]
names = [detector.getName() for detector in camera]
#LSST bandpasses
LSST_BPass = BandpassDict.loadTotalBandpassesFromFiles()

In [4]:
# visits database
dbname = '/global/projecta/projectdirs/lsst/groups/SSim/DC2/minion_1016_desc_dithered_v4.db'
ObsMetaData = ObservationMetaDataGenerator(database=dbname)

# SNe database to make light curves from LC parameters rather than using truth table
database = '/global/cscratch1/sd/bos0109/sne_params_cosmoDC2_v1.1_181121.db'


In [5]:
# Create a template for queries
conn = sqlite3.connect(database)
c = conn.cursor()
query_tmpl = "SELECT * FROM sne_params WHERE snra_in > {} AND snra_in < {} "
query_tmpl+= "AND sndec_in > {} AND sndec_in < {}"

In [ ]:
def main(ramax=58, ramin=56, decmin=-32, decmax=-31, t0=59215, tm=61406):
    '''Brunos function to pull the SNe from within the time and ra, dec ranges'''
    query = query_tmpl.format(ramin, ramax, decmin, decmax)

    sntab = pd.read_sql_query(query, conn)
    #sntab.to_csv('./catalogs+tables/sn_cat_rectangle.csv')

    #if os.path.isfile('./catalogs+tables/full_t_visits_from_minion.csv'):
    #    visitab = pd.read_csv('./catalogs+tables/full_t_visits_from_minion.csv')
    #else:
    res = ObsMetaData.getObservationMetaData(boundLength=2, boundType='circle', 
                                        fieldRA=(ramin-3, ramax+3), 
                                        fieldDec=(decmin-3, decmax+3), 
                                        expMJD=(t0, tm))
    parsed = [Odict(obsmd.summary['OpsimMetaData']) for obsmd in res]
    print(len(parsed), 'is this how many you want?')
    for obsmd, summ in zip(res, parsed):
        ditherRa = np.rad2deg(summ['descDitheredRA'])
        ditherDec = np.rad2deg(summ['descDitheredDec'])
        ditherRot = np.rad2deg(summ['descDitheredRotTelPos'])
        summ['descDitheredRotSkyPos'] =  getRotSkyPos(ditherRa, ditherDec, 
                                                        obsmd, ditherRot)

    df = pd.DataFrame(parsed)
    df = df[df['filter'].isin(('g', 'r', 'i', 'z'))]

    X = df[['obsHistID', 'filter', 'FWHMeff', 'descDitheredRA', 
            'descDitheredDec', 'descDitheredRotTelPos', 'airmass', 
            'fiveSigmaDepth', 'expMJD', 'descDitheredRotSkyPos',
            'fieldRA', 'fieldDec', 'rotSkyPos', 'rotTelPos']].copy()
    X.descDitheredRA = np.rad2deg(X.descDitheredRA)
    X.descDitheredDec = np.rad2deg(X.descDitheredDec)
    X.descDitheredRotTelPos = np.rad2deg(X.descDitheredRotTelPos)
    #X.descDitheredRotSkyPos = np.rad2deg(X.descDitheredRotSkyPos) already in deg

    X.fieldRA = np.rad2deg(X.fieldRA)
    X.fieldDec = np.rad2deg(X.fieldDec)
    X.rotTelPos = np.rad2deg(X.rotTelPos)
    X.rotSkyPos = np.rad2deg(X.rotSkyPos)

    X['d1'] = angularSeparation(ramin, decmax, 
        X.descDitheredRA.values, X.descDitheredDec.values)
    X['d2'] = angularSeparation(ramin, decmin, 
        X.descDitheredRA.values, X.descDitheredDec.values)
    X['d3'] = angularSeparation(ramax, decmax, 
        X.descDitheredRA.values, X.descDitheredDec.values)
    X['d4'] = angularSeparation(ramax, decmin, 
        X.descDitheredRA.values, X.descDitheredDec.values)
    visitab = X.query('d1 < 1.75 | d2 < 1.75 | d3 < 1.75 |d4 < 1.75')
    del(X)
    del(df)
    visitab.to_csv('./catalogs+tables/full_t_visits_from_minion.csv')
    # setting the observation telescope status
    boresight = []
    orientation = []
    wcs_list = []
    for avisit in visitab.itertuples():
        bsight = geom.SpherePoint(avisit.descDitheredRA*geom.degrees, 
                                  avisit.descDitheredDec*geom.degrees)
        orient = (90-avisit.descDitheredRotSkyPos)*geom.degrees

        wcs_list.append([makeSkyWcs(t, orient, flipX=False, boresight=bsight,
                       projection='TAN') for t in trans])
        orientation.append(orient)
        boresight.append(bsight)
    
    times = visitab['expMJD']
    bands = visitab['filter']
    depths= visitab['fiveSigmaDepth']
    #colnames = ['mjd', 'filter']
    data_cols = {'mjd': times, 'filter': bands, 'visitn': visitab['obsHistID']}
    n_observ = []; n_trueobserv = []
    
    for asn in sntab.itertuples():
        sn_mod = SNObject(ra=asn.snra_in, dec=asn.sndec_in)
        sn_mod.set(z=asn.z_in, t0=asn.t0_in, x1=asn.x1_in, 
                   c=asn.c_in, x0=asn.x0_in)

        sn_skyp = afwGeom.SpherePoint(asn.snra_in, asn.sndec_in, afwGeom.degrees)
    
        size = len(times)
        sn_flxs = np.zeros(size); sn_mags = np.zeros(size); 
        sn_flxe = np.zeros(size); sn_mage = np.zeros(size); 
        sn_obsrvd = []; sn_observable = []
        ii = 0
        for mjd, filt, wcsl, m5 in zip(times, bands, wcs_list, depths):
            flux = sn_mod.catsimBandFlux(mjd, LSST_BPass[filt])
            mag = sn_mod.catsimBandMag(LSST_BPass[filt], mjd, flux)
            flux_er = sn_mod.catsimBandFluxError(mjd, LSST_BPass[filt], m5, flux)
            mag_er = sn_mod.catsimBandMagError(mjd, LSST_BPass[filt], m5, magnitude=mag)
            
            # checking sensors containing this object
            contain = [box.contains(afwGeom.Point2I(wcs.skyToPixel(sn_skyp))) \
                           for box, wcs in zip(boxes, wcsl)]
            observed = np.sum(contain) > 0
            observable = observed & (mag + mag_er < 27.0) & (mag_er < 0.5)
            # if observed:
            #     print('Overlaps ccd', names[np.where(contain)[0][0]])      
            sn_observable.append(observable)      
            sn_obsrvd.append(observed)
            sn_flxs[ii] = flux  # done
            sn_mags[ii] = mag
            sn_flxe[ii] = flux_er
            sn_mage[ii] = mag_er
            ii+=1
        
        data_cols[asn.snid_in+'_observable'] = sn_observable
        data_cols[asn.snid_in+'_observed'] = sn_obsrvd
        data_cols[asn.snid_in+'_flux'] = sn_flxs
        data_cols[asn.snid_in+'_fluxErr'] = sn_flxe
        data_cols[asn.snid_in+'_mag'] = sn_mags
        data_cols[asn.snid_in+'_magErr'] = sn_mage
        n_observ.append(np.sum(sn_obsrvd))
        n_trueobserv.append(np.sum(sn_observable))
    sntab['Nobserv'] = n_observ
    sntab['N_trueobserv'] = n_trueobserv
    lightcurves = pd.DataFrame(data_cols)
    
    dest_lc = './lightcurves/lightcurves_cat_rect_{}_{}_{}_{}.csv'
    lightcurves.to_csv(dest_lc.format(ramax, ramin, decmax, decmin))
    dest_snfile = './catalogs+tables/supernovae_cat_rect_{}_{}_{}_{}.csv'
    sntab.to_csv(dest_snfile.format(ramax, ramin, decmax, decmin))
    print("""Stored the lightcurves in {}, 
             the SN catalog in {}""".format(
                dest_lc.format(ramax, ramin, decmax, decmin), 
                dest_snfile.format(ramax, ramin, decmax, decmin)))
    return data_cols




In [ ]:
max_ra = 53.00727812651823+0.01
min_ra = 53.00727812651823 - 0.01
max_dec = -28.387552632959427 + 0.01
min_dec =  -28.387552632959427 - 0.01
#52.93572544959448 54.72776196140405 -29.78294629153197 -28.2276731919467
print( 'boundaries for diff catalog tract:', min_ra, max_ra, min_dec, max_dec)
lc_dia = main(ramax=max_ra, ramin=min_ra, decmin=min_dec, decmax=max_dec, t0=59215, tm=61406)

In [9]:
print(np.shape(lc_dia))

NameError: name 'lc_dia' is not defined